In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import html5lib
import requests
import json

In [2]:
demo = "YOUR API KEY"

tickers = ['SPY','DAL','TM','BAC','MTB','BK','BMY',
           'GLD','KO','GL','PG','KR','WMT',
           'ERIC','MDT','JNJ','ABNB','AAPL']
data={}
back_test_data_50_100 = {}
back_test_data_50_200 = {}
wback_test_data_50_100 = {}
wback_test_data_50_200 = {}
results = pd.DataFrame(columns=['Ticker','RSI','MFI','Bollinger Bands Ratio','MACD - Signal Line EWM9',
                                'Invested MA50-MA100','Invested MA50-MA200',
                                'Invested WMA50-WMA100','Invested WMA50-WMA200','Normalized Cumulative Relative Change'])

In [3]:
def get_data(ticker):
    
    global data
    global back_test_data_50_100 
    global back_test_data_50_200 
    global wback_test_data_50_100
    global wback_test_data_50_200
    
#try:
    # Get Daily Stock Price Data
    # Transform the Json String to Pandas DataFrame

    stocks = requests.get(f'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?apikey={demo}').json()
    stocks = stocks['historical']
    stock_prices_data = pd.DataFrame.from_dict(stocks)
    data[ticker] = stock_prices_data
    data[ticker]['ticker']=ticker

    # Sort Values by Date
    data[ticker] = data[ticker][.sort_values(by="date")]

    # Set Date as Index 
    data[ticker] = data[ticker].set_index('date')    

    #Create DFs for BackTesting
    # SMA
    back_test_data_50_100[ticker] = data[ticker].drop(columns = ['open','high','low','close','adjClose','volume',
                                                                 'unadjustedVolume','change','changePercent','vwap',
                                                                 'label','changeOverTime'])

    back_test_data_50_200[ticker] = data[ticker].drop(columns = ['open','high','low','close','adjClose','volume',
                                                                 'unadjustedVolume','change','changePercent','vwap',
                                                                 'label','changeOverTime'])

    # WMA
    wback_test_data_50_100[ticker] = data[ticker].drop(columns = ['open','high','low','close','adjClose','volume',
                                                                  'unadjustedVolume','change','changePercent','vwap',
                                                                  'label','changeOverTime'])

    wback_test_data_50_200[ticker] = data[ticker].drop(columns = ['open','high','low','close','adjClose','volume',
                                                                  'unadjustedVolume','change','changePercent','vwap',
                                                                  'label','changeOverTime'])

    #print('get_data_function_completed for:' + ticker)

#except:

    #print('get_data_function_NOT_completed for:' + ticker)

In [4]:
def returns(ticker):

    try:
        ######################## Price Change ###############################################################
        
        # Nominal Absolute Change
        data[ticker]['Absolute Price Change'] = data[ticker]['close'].diff(1)
        
        # Normalized Absolute Change
        data[ticker]['Normalized Absolute Price Change'] = data[ticker]['adjClose'].diff(1)
        
        # Nominal Relative Change
        data[ticker]['Close Price Pt-1'] = data[ticker]['close'].shift(1)
        data[ticker]['Close Price Change Pt/Pt-1'] = data[ticker]['close']/data[ticker]['Close Price Pt-1']
    
        # Normalized Relative Change
        data[ticker]['Adj. Close Price Pt-1'] = data[ticker]['adjClose'].shift(1)
        data[ticker]['Adj. Close Price Change Pt/Pt-1'] =  data[ticker]['adjClose']/data[ticker]['Adj. Close Price Pt-1']
    
        ######################## Daily Data #################################################################
    
        # Daily Returns in % and Cumulative Return in %
        data[ticker]['Daily Returns'] = data[ticker]['close'].pct_change(1)
        data[ticker]['Daily Cumulative Return'] = (1+ data[ticker]['Daily Returns']).cumprod()
    
        # Daily Normalized Returns in % and Normalized  Cumulative Return in %
        
        data[ticker]['Daily Normalized Returns'] = data[ticker]['adjClose'].pct_change(1)
        data[ticker]['Daily Normalized Cumulative Return'] = (1+ data[ticker]['Daily Normalized Returns']).cumprod()
    
        
        print('returns_function_completed for:' + ticker)
        
    except:
        
        print('returns_function_NOT_completed for:' + ticker)

In [5]:
def sma (ticker):
    
    try:
        # Simple Moving Averages
        # MA 50, 100, 200 Are Used for Crossovers, Golden Cross or Death Cross 
        data[ticker]['MA50'] = data[ticker]['close'].rolling(50).mean()
        data[ticker]['MA100'] = data[ticker]['close'].rolling(100).mean()
        data[ticker]['MA120'] = data[ticker]['close'].rolling(120).mean()
        data[ticker]['MA200'] = data[ticker]['close'].rolling(200).mean()
    
        print('sma_function_completed for:' + ticker)
        
    except:
        
        print('sma_function_NOT_completed for:' + ticker)

In [6]:
def ewma (ticker):
    
    try:
        # Exponential Moving Averages
        # EMA 50, 100, 200 Are Used for Crossovers, Golden Cross or Death Cross 
        data[ticker]['WMA50'] = data[ticker]['close'].ewm(50).mean()
        data[ticker]['WMA100'] = data[ticker]['close'].ewm(100).mean()
        data[ticker]['WMA120'] = data[ticker]['close'].ewm(120).mean()
        data[ticker]['WMA200'] = data[ticker]['close'].ewm(200).mean()   
        
        print('ewma_function_completed for:' + ticker)
        
    except:
        
        print('ewma_function_NOT_completed for:' + ticker)

In [7]:
def macd (ticker):
    
    try:
        ## Moving average convergence divergence (MACD) ##
        data[ticker]['WMA12'] = data[ticker]['close'].ewm(12,adjust=False).mean()
        data[ticker]['WMA26'] = data[ticker]['close'].ewm(26,adjust=False).mean()
        data[ticker]['MACD'] = data[ticker]['WMA12'] - data[ticker]['WMA26']
        data[ticker]['Signal Line EWM9'] = data[ticker]['MACD'].ewm(9,adjust=False).mean()
        
        # Difference between  MACD and Signal Line
        data[ticker]['MACD - Signal Line EWM9'] = data[ticker]['MACD']-data[ticker]['Signal Line EWM9']
        
        print('macd_function_completed for:' + ticker)
        
    except:
        
        print('macd_function_NOT_completed for:' + ticker)

In [8]:
def bollinger_bands (ticker):    
     
    try:
        data[ticker]['MA20'] = data[ticker]['close'].rolling(20).mean()
        data[ticker]['Upper Band'] = data[ticker]['MA20'] + 2*data[ticker]['close'].rolling(window=20).std()
        data[ticker]['Lower Band'] = data[ticker]['MA20'] - 2*data[ticker]['close'].rolling(window=20).std()
        data[ticker]['Std20'] = data[ticker]['close'].rolling(window=20).std()
        data[ticker]['Bollinger Bands Ratio'] = (data[ticker]['Std20']/data[ticker]['MA20']) * 100
       
        print('bollinger_bands_function_completed for:' + ticker)
        
    except:
        
        print('bollinger_bands_function_NOT_completed for:' + ticker)

In [9]:
def crossover_sma (ticker):
  
    try:
        
        # SMA Crossover SMA 50 > 100 Golden Cross SMA 50 < 100 Death Cross
        data[ticker]['Crossover MA50-100'] = [1 if data[ticker].loc[i,'MA50'] > data[ticker].loc[i,'MA100']
                                        else 0 for i in data[ticker].index]
    
        # SMA Backtesting Investment Strategy SMA 50-100 Cumulative Returns
        back_test_data_50_100[ticker]['Daily Cumulative Return MA50-100'] = np.cumprod(data[ticker][data[ticker]['Crossover MA50-100'] == 1]
                                                                                       [['Close Price Change Pt/Pt-1']])

    
        # SMA Crossover SMA 50 > 200 Golden Cross SMA 50 < 200 Death Cross
        data[ticker]['Crossover MA50-200'] = [1 if data[ticker].loc[i,'MA50'] > data[ticker].loc[i,'MA200']
                                       else 0 for i in data[ticker].index]
    
        # SMA Backtesting Investment Strategy SMA 50-200 Cumulative Returns
        back_test_data_50_200[ticker]['Daily Cumulative Return MA50-200'] = np.cumprod(data[ticker][data[ticker]['Crossover MA50-200'] == 1]
                                                                                       [['Close Price Change Pt/Pt-1']])
    
        # Buy and Hold Daily Cumulative Return
        back_test_data_50_100[ticker]['Buy and Hold Daily Cumulative Return'] = np.cumprod(data[ticker]['Close Price Change Pt/Pt-1'])
        back_test_data_50_200[ticker]['Buy and Hold Daily Cumulative Return'] = np.cumprod(data[ticker]['Close Price Change Pt/Pt-1'])
    
        print('crossover_sma_function_completed for:' + ticker)
        
    except:
        
        print('crossover_sma_function_NOT_completed for:' + ticker)

In [10]:
def crossover_ewma (ticker):
    
    try:
        
        # WMA Crossover SMA 50 > 100 Golden Cross WMA 50 < 100 Death Cross
        data[ticker]['Crossover WMA50-100'] = [1 if data[ticker].loc[i,'WMA50'] > data[ticker].loc[i,'WMA100']
                                         else 0 for i in data[ticker].index]
    
        # WMA Backtesting Investment Strategy WMA 50-100 Cumulative Returns
        wback_test_data_50_100[ticker]['Daily Cumulative Return WMA50-100'] = np.cumprod(data[ticker][data[ticker]['Crossover WMA50-100'] == 1]
                                                                                         [['Close Price Change Pt/Pt-1']])

    
        # WMA Crossover WMA 50 > 200 Golden Cross WMA 50 < 200 Death Cross
        data[ticker]['Crossover WMA50-200'] = [1 if data[ticker].loc[i,'WMA50'] > data[ticker].loc[i,'WMA200']
                                         else 0 for i in data[ticker].index]
    
        # WMA Backtesting Investment Strategy WMA 50-200 Cumulative Returns
        wback_test_data_50_200[ticker]['Daily Cumulative Return WMA50-200'] = np.cumprod(data[ticker][data[ticker]['Crossover WMA50-200'] == 1]
                                                                                         [['Close Price Change Pt/Pt-1']])
    
        # Buy and Hold Daily Cumulative Return
        wback_test_data_50_100[ticker]['Buy and Hold Daily Cumulative Return'] = np.cumprod(data[ticker]['Close Price Change Pt/Pt-1'])
        wback_test_data_50_200[ticker]['Buy and Hold Daily Cumulative Return'] = np.cumprod(data[ticker]['Close Price Change Pt/Pt-1'])
    
        print('crossover_ewma_function_completed for:' + ticker)
        
    except:
        
        print('crossover_ewma_function_NOT_completed for:' + ticker) 

In [11]:
def rsi (ticker):
    
    # Relative Strength Index (RSI)
    
    try:
        
        data[ticker]['Up'] = [1 if data[ticker].loc[i,'Absolute Price Change'] > 0 else 0 for i in data[ticker].index]
        data[ticker]['Down'] = [1 if data[ticker].loc[i,'Absolute Price Change'] < 0 else 0 for i in data[ticker].index]

        data[ticker]['Positive movement'] = data[ticker]['Up'] * data[ticker]['Normalized Absolute Price Change']
        data[ticker]['Negative movement'] = data[ticker]['Down'] * data[ticker]['Normalized Absolute Price Change'] * -1 # (-1) because we need to have positve values

        data[ticker]['Avg gain'] = data[ticker]['Positive movement'].rolling(14).mean()
        data[ticker]['Avg loss'] = data[ticker]['Negative movement'].rolling(14).mean()
        
        data[ticker]['RS'] = data[ticker]['Avg gain']/data[ticker]['Avg loss']
        data[ticker]['RSI']  = (100 - (100 / (1 + data[ticker]['RS'])))  
        
        print('rsi_function_completed for:' + ticker)
        
    except:
        
        print('rsi_function_NOT_completed for:' + ticker) 

In [12]:
def mfi (ticker):
    
    # Money Flow Index - MFI
    
    try:
        
        # Typical Price
        data[ticker]['Typical Price'] = (data[ticker]['high'] + data[ticker]['low'] + data[ticker]['close'])/3
     
        # Raw Money Flow 
        data[ticker]['Raw Money Flow'] = (data[ticker]['Typical Price'] * data[ticker]['volume']).diff(1)
        
        ############################## # Money Flow Ratio # ######################################################
        
        # Positive_Money_Flow
        data[ticker]['Positive Money Movement'] = [1 if data[ticker].loc[i,'Raw Money Flow'] > 0 else 0 for i in data[ticker].index]
        
        ## Period_Negative_Money_Flow
        data[ticker]['Negative Money Movement'] = [1 if data[ticker].loc[i,'Raw Money Flow'] < 0 else 0 for i in data[ticker].index]
        
        ## 14_Period_Positive_Money_Flow
        data[ticker]['Positive Money Flow'] = data[ticker]['Positive Money Movement'] * data[ticker]['Raw Money Flow']
        data[ticker]['14 Period Positive Money Flow'] = data[ticker]['Positive Money Flow'].rolling(14).mean()
        
        ### 14_Period_Negative_Money_Flow
        data[ticker]['Negative Money Flow'] = data[ticker]['Negative Money Movement'] * (data[ticker]['Raw Money Flow'] * (-1))
        data[ticker]['14 Period Negative Money Flow'] = data[ticker]['Negative Money Flow'].rolling(14).mean()
        
        #### Money Flow Ratio
        data[ticker]['Money Flow Ratio'] = data[ticker]['14 Period Positive Money Flow'] / data[ticker]['14 Period Negative Money Flow']
        
        ############################### # Money Flow Index # ##########################################################
        
        data[ticker]['MFI'] = (100 - (100 / (1 + data[ticker]['Money Flow Ratio'])))
       
        print('mfi_function_completed for:' + ticker)
        
    except:
        
        print('mfi_function_NOT_completed for:' + ticker)        

In [13]:
# Get Stock Price data 
# Technical Analysis of Stock price data

for ticker in tickers:
    try:
        
        get_data(ticker)
        returns(ticker)
        sma(ticker)
        ewma(ticker)
        macd(ticker)
        bollinger_bands(ticker)
        crossover_sma(ticker)
        crossover_ewma(ticker)
        rsi(ticker)
        mfi(ticker)
        
        new_row = {'Ticker':ticker,
                'RSI':data[ticker]['RSI'].iloc[-1],
                'MFI':data[ticker]['MFI'].iloc[-1],
                'Bollinger Bands Ratio':data[ticker]['Bollinger Bands Ratio'].iloc[-1],
                'MACD - Signal Line EWM9':data[ticker]['MACD - Signal Line EWM9'].iloc[-1],
                'Invested MA50-MA100':back_test_data_50_100[ticker]['Daily Cumulative Return MA50-100'].iloc[-1],
                'Invested MA50-MA200':back_test_data_50_200[ticker]['Daily Cumulative Return MA50-200'].iloc[-1],
                'Invested WMA50-WMA100': wback_test_data_50_100[ticker]['Daily Cumulative Return WMA50-100'].iloc[-1],
                'Invested WMA50-WMA200':wback_test_data_50_200[ticker]['Daily Cumulative Return WMA50-200'].iloc[-1],
                'Normalized Cumulative Relative Change':data[ticker]['Daily Cumulative Return'].iloc[-1]} 
        results = results.append(new_row, ignore_index=True)
        
                   
        print(ticker + ":Added")
        
    except:
        
        print(ticker + ":Something went wrong")

returns_function_completed for:SPY
sma_function_completed for:SPY
ewma_function_completed for:SPY
macd_function_completed for:SPY
bollinger_bands_function_completed for:SPY
crossover_sma_function_completed for:SPY
crossover_ewma_function_completed for:SPY
rsi_function_completed for:SPY
mfi_function_completed for:SPY
returns_function_completed for:DAL
sma_function_completed for:DAL
ewma_function_completed for:DAL
macd_function_completed for:DAL
bollinger_bands_function_completed for:DAL
crossover_sma_function_completed for:DAL
crossover_ewma_function_completed for:DAL
rsi_function_completed for:DAL
mfi_function_completed for:DAL
returns_function_completed for:TM
sma_function_completed for:TM
ewma_function_completed for:TM
macd_function_completed for:TM
bollinger_bands_function_completed for:TM
crossover_sma_function_completed for:TM
crossover_ewma_function_completed for:TM
rsi_function_completed for:TM
mfi_function_completed for:TM
returns_function_completed for:BAC
sma_function_complet

In [14]:
results

,Ticker,RSI,MFI,Bollinger Bands Ratio,MACD - Signal Line EWM9,Invested MA50-MA100,Invested MA50-MA200,Invested WMA50-WMA100,Invested WMA50-WMA200,Normalized Cumulative Relative Change
0,SPY,55.525868,50.661698,0.955772,0.277107,1.402046,1.401276,1.436134,1.900075,2.027572
1,DAL,44.033233,48.474454,3.446750,-0.372039,NaN,0.699171,1.102491,0.801488,1.062792
2,TM,57.254290,51.331973,1.044924,-0.786087,1.375935,1.340766,1.350902,1.201695,1.613819
3,BAC,40.575080,48.773924,3.556230,-0.509749,0.887339,0.983063,1.524554,1.542049,2.596036
4,MTB,40.258398,53.071829,4.174717,-1.974668,NaN,0.653849,1.019800,0.956034,1.165579
5,BK,53.423339,61.356130,1.953580,-0.208086,0.817633,0.897119,1.265430,1.252160,1.256950
6,BMY,52.864157,53.936550,0.825956,0.006397,0.798047,1.067839,0.873361,0.883543,0.882568
7,GLD,47.777778,46.989668,0.927357,-0.229854,1.185425,1.392593,1.221066,1.290789,1.315629
8,KO,80.000000,49.359089,2.228690,0.191309,0.960104,1.037697,1.013544,1.064554,1.319355
9,GL,46.279492,45.029593,1.415382,-0.505353,NaN,0.874611,0.840132,0.919004,1.521335
